In [52]:
import pandas as pd
import dbio  # private package
from IPython.display import Markdown

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.prompts.prompt import PromptTemplate
from langchain import SQLDatabase, SQLDatabaseChain
from langchain.chat_models import ChatOpenAI

# First pass:

Put everything in one table for the initial prototype.

Didn't work well.  See below for Second Pass.

In [ ]:
# columns = Item type, Prices (Large, Medium, Small), Description
menu = {'Pepperoni pizza': ['pizza', '12.95, 10, 7', 'Our most popular'],
        'Cheese pizza':  ['pizza', '10.95, 9.25, 6.50', 'Extreme cheesiness'],
        'Eggplant pizza':  ['pizza', '11.95, 9.75, 6.75', 'A contemporary classic'],
        'Fries': ['side item', '4.50, 3.50', 'Available for a limited time'],
        'Greek salad': ['side item', '7.25', 'Positively pantheonic'],
        'Extra cheese': ['topping', '2.00', ''],
        'Mushrooms': ['topping', '1.50', 'Non-hallucinogenic'],
        'Sausage': ['topping', '3.00', 'Spicy with a hint of sweet'],
        'Canadian bacon': ['topping', '3.50', ''],
        'AI sauce': ['topping', '1.50', 'No human would have made it this way'],
        'Peppers': ['topping', '1.00', ''],
        'Coke': ['drink', '3.00, 2.00, 1.00', '3 Liter, 2 Liter, 16 ounces'],
        'Sprite': ['drink', '3.00, 2.00, 1.00', '3 Liter, 2 Liter, 16 ounces'],
        'Bottled water': ['drink', '5.00', '20 ounces'],
        }

menu_df = pd.DataFrame(menu).T.reset_index()
menu_df.columns = ['Item', 'Type', 'Prices (Large, Medium, Small)', 'Description']
menu_df

In [ ]:
# store table in pizza_v1.db
cobj = dbio.connectors.SQLite('pizza_v1.db')
cobj.write(menu_df, 'menu', if_exists='replace')

cobj.read('select count(*) from menu')

In [ ]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///./pizza_v1.db")
llm = OpenAI(model_name="gpt-3.5-turbo"temperature=0, verbose=True)

In [ ]:
# database chain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
# query

db_chain.run("How many kinds of pizza do you offer?")

In [ ]:
# query - drinks

db_chain.run("What drinks can I order")

In [ ]:
# query - bottled water

db_chain.run("lease tell me how many sizes of bottled water \
                       you have. Do you know how big they are?")

Try again with updated prompt:
 - check in greater 1 field
 - lower case text

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, 
then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

When looking up information on products, be sure to check the Item and Type columns. Also lowercase the user
input and fields in the tables to ensure a match. Use the LIKE operator to allow for partial string matches.

In the WHERE CLAUSE combine mutliple filters together with a logical OR operator.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Step 1:
Given an input question, write the syntactically correct {dialect} query that meets the following guidelines:
  a. Be sure to check the the Item, Type, and Description columns.
  b. Combine multiple WHERE filters with a logical OR
  c. Lowercase the user input

Step 2: Run the query.

Step 3: Examine the results and return the answer.

Use the following format:
Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

Table schema explanation:
Item - the product name
Type - kind of item (pizza, side item, topping, drink)
Price - comma separate list of prices for large, medium, small.
Description - extra information including sizes

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)

In [ ]:
db_chain.run("Please tell me how many sizes of water containing items do\
                       you have. Do you know how big they are?")

In [ ]:
db_chain.run("How much is a large cheese pizza?")

# Second Pass: tables for each kind of item

In [54]:
# columns = Large, Medium, Small, Description
menu_pizzas = {'Pepperoni pizza': ['12.95', '10', '7', 'Our most popular'],
        'Cheese pizza':  ['10.95', '9.25', '6.50', 'Extreme cheesiness'],
        'Eggplant pizza':  ['11.95', '9.75', '6.75', 'A contemporary classic'],
        }

menu_sides = { # Item, Large, Medium, Small, Description
        'Fries': ['4.50', '3.50', '', 'Available for a limited time'],
        'Greek salad': ['7.25', '', '', 'Positively pantheonic'],
        }

menu_toppings = { # Topping price
        'Extra cheese': ['2.00', ''],
        'Mushrooms': [ '1.50', 'Non-hallucinogenic'],
        'Sausage': ['3.00', 'Spicy with a hint of sweet'],
        'Canadian bacon': ['3.50', ''],
        'AI sauce': ['1.50', 'No human would have made it this way'],
        'Peppers': ['1.00', ''],
        }

menu_drinks = { # Price_large, Price_medium, Price_small, Description
        'Coke': ['3.00', '2.00', '1.00', 'Large = 3 Liter, Medium = 2 Liter, Smal = 16 ounces'],
        'Sprite': ['3.00', '2.00', '1.00', 'Large = 3 Liter, Medium = 2 Liter, Smal = 16 ounces'],
        'Bottled water': ['5.00', None, None, 'Single size = 20 ounces'],
        }

menu_pizzas_df = pd.DataFrame(menu_pizzas).T.reset_index()
menu_pizzas_df.columns = ['Item', 'Price_Large', 'Price_Medium', 'Price_Small', 'Description']

menu_sides_df = pd.DataFrame(menu_sides).T.reset_index()
menu_sides_df.columns = ['Item', 'Price_Large', 'Price_Medium', 'Price_Small', 'Description']

menu_drinks_df = pd.DataFrame(menu_drinks).T.reset_index()
menu_drinks_df.columns = ['Item', 'Price_Large', 'Price_Medium', 'Price_Small', 'Description']

menu_toppings_df = pd.DataFrame(menu_toppings).T.reset_index()
menu_toppings_df.columns = ['Item', 'Price', 'Description']

display(Markdown(f'<span style="color: #ff0000">pizzas:</span>'))
display(menu_pizzas_df.head())
display(Markdown(f'<span style="color: #ff0000">sides:</span>'))
display(menu_sides_df.head())
display(Markdown(f'<span style="color: #ff0000">toppings:</span>'))
display(menu_toppings_df.head())
display(Markdown(f'<span style="color: #ff0000">drinks:</span>'))
display(menu_drinks_df.head())

<span style="color: #ff0000">pizzas:</span>

,Item,Price_Large,Price_Medium,Price_Small,Description
0,Pepperoni pizza,12.95,10,7,Our most popular
1,Cheese pizza,10.95,9.25,6.50,Extreme cheesiness
2,Eggplant pizza,11.95,9.75,6.75,A contemporary classic


<span style="color: #ff0000">sides:</span>

,Item,Price_Large,Price_Medium,Price_Small,Description
0,Fries,4.50,3.50,,Available for a limited time
1,Greek salad,7.25,,,Positively pantheonic


<span style="color: #ff0000">toppings:</span>

,Item,Price,Description
0,Extra cheese,2.00,
1,Mushrooms,1.50,Non-hallucinogenic
2,Sausage,3.00,Spicy with a hint of sweet
3,Canadian bacon,3.50,
4,AI sauce,1.50,No human would have made it this way


<span style="color: #ff0000">drinks:</span>

,Item,Price_Large,Price_Medium,Price_Small,Description
0,Coke,3.00,2.00,1.00,"Large = 3 Liter, Medium = 2 Liter, Smal = 16 o..."
1,Sprite,3.00,2.00,1.00,"Large = 3 Liter, Medium = 2 Liter, Smal = 16 o..."
2,Bottled water,5.00,None,None,Single size = 20 ounces


In [ ]:
# store table in pizza_v1.db
cobj = dbio.connectors.SQLite('pizza_v2.db')
cobj.write(menu_pizzas_df, 'pizzas', if_exists='replace')
cobj.write(menu_toppings_df, 'toppings', if_exists='replace')
cobj.write(menu_sides_df, 'sides', if_exists='replace')
cobj.write(menu_drinks_df, 'drinks', if_exists='replace')

cobj.read('select count(*) from pizzas')

In [5]:
'''
# custom prompt for SQLDatabaseChain
_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, 
then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

When looking up information on products, be sure to check the Item and Type columns. Also lowercase the user
input and fields in the tables to ensure a match. Use the LIKE operator to allow for partial string matches.

In the WHERE CLAUSE combine mutliple filters together with a logical OR operator.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
'''

In [9]:
# construct chain from custom prompt, llm, and database
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", temperature=0, verbose=True)
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)  # did not use custom prompt in the end

#db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)  # if using custom prompt

In [47]:
# default db_chain prompt template
print((db_chain.llm_chain.prompt.template))

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

In [41]:
# query - pizza sizes - prior hallucination
result = db_chain.run("Please tell me how many sizes of pizza you have and how big they are.")
result



> Entering new  chain...
Please tell me how many sizes of pizza you have and how big they are.
SQLQuery:SELECT "Item", "Description" FROM pizzas
SQLResult: [('Pepperoni pizza', 'Our most popular'), ('Cheese pizza', 'Extreme cheesiness'), ('Eggplant pizza', 'A contemporary classic')]
Answer:There are three sizes of pizza: Large, Medium, and Small. The sizes and their corresponding dimensions are not provided in the database.
> Finished chain.


'There are three sizes of pizza: Large, Medium, and Small. The sizes and their corresponding dimensions are not provided in the database.'

In [11]:
# query - bottled water - prior hallucination
db_chain.run("Please tell me how many sizes of bottled water \
                       you have. Do you know how big they are?")



> Entering new  chain...
Please tell me how many sizes of bottled water                        you have. Do you know how big they are?
SQLQuery:SELECT COUNT(*) FROM drinks WHERE Item = "Bottled water"
SQLResult: [(1,)]
Answer:There is only one size of bottled water available, which is a single size of 20 ounces.
> Finished chain.


'There is only one size of bottled water available, which is a single size of 20 ounces.'

In [12]:
# query - greek salad - prior hallucination
db_chain.run("Is it possible to top the cheese pizza with the greek salad?")



> Entering new  chain...
Is it possible to top the cheese pizza with the greek salad?
SQLQuery:SELECT pizzas.Item, toppings.Item AS Topping, toppings.Price
FROM pizzas
JOIN toppings ON pizzas.Item = 'Cheese pizza' AND toppings.Item = 'Greek salad'
SQLResult: []
Answer:No, it is not possible to top the cheese pizza with the Greek salad as there are no results returned from the query.
> Finished chain.


'No, it is not possible to top the cheese pizza with the Greek salad as there are no results returned from the query.'

In [13]:
# query - drinks
db_chain.run("What drinks can I order?")



> Entering new  chain...
What drinks can I order?
SQLQuery:SELECT "Item", "Price_Large", "Price_Medium", "Price_Small" FROM drinks LIMIT 5
SQLResult: [('Coke', '3.00', '2.00', '1.00'), ('Sprite', '3.00', '2.00', '1.00'), ('Bottled water', '5.00', None, None)]
Answer:You can order Coke, Sprite, and Bottled water.
> Finished chain.


'You can order Coke, Sprite, and Bottled water.'

In [16]:
db_chain.run("Are the mushroom toppings hallucinogenic?")



> Entering new  chain...
Are the mushroom toppings hallucinogenic?
SQLQuery:SELECT "Description" FROM toppings WHERE "Item" = 'Mushrooms'
SQLResult: [('Non-hallucinogenic',)]
Answer:No, the mushroom toppings are not hallucinogenic.
> Finished chain.


'No, the mushroom toppings are not hallucinogenic.'

In [17]:
db_chain.run("What is the price of a cheese pizza with sausage?")



> Entering new  chain...
What is the price of a cheese pizza with sausage?
SQLQuery:SELECT pizzas."Price_Small" FROM pizzas JOIN toppings ON pizzas.Item = toppings.Item WHERE pizzas.Item = "Cheese pizza" AND toppings.Item = "Sausage"
SQLResult: []
Answer:There is no result for the given query. It seems that there is no small size for the Cheese pizza with sausage topping.
> Finished chain.


'There is no result for the given query. It seems that there is no small size for the Cheese pizza with sausage topping.'

In [19]:
db_chain.run("What is the price of a small cheese pizza topped with both sausage and mushroom?")



> Entering new  chain...
What is the price of a small cheese pizza topped with both sausage and mushroom?
SQLQuery:SELECT "Price_Small" + SUM("Price") AS "Total Price" FROM pizzas JOIN toppings ON pizzas.Item = toppings.Item WHERE pizzas.Item = "Cheese pizza" AND toppings.Item IN ("Sausage", "Mushrooms")
SQLResult: [(None,)]
Answer:There is no price available for a small cheese pizza topped with both sausage and mushroom.
> Finished chain.


'There is no price available for a small cheese pizza topped with both sausage and mushroom.'

In [20]:
db_chain.run("What is the diameter of each pizza size?")



> Entering new  chain...
What is the diameter of each pizza size?
SQLQuery:SELECT "Item", "Description", 
CASE 
    WHEN "Price_Large" IS NOT NULL THEN "Price_Large" 
    WHEN "Price_Medium" IS NOT NULL THEN "Price_Medium" 
    ELSE "Price_Small" 
END AS "Price",
CASE 
    WHEN "Price_Large" IS NOT NULL THEN 16 
    WHEN "Price_Medium" IS NOT NULL THEN 12 
    ELSE 8 
END AS "Diameter"
FROM pizzas
SQLResult: [('Pepperoni pizza', 'Our most popular', '12.95', 16), ('Cheese pizza', 'Extreme cheesiness', '10.95', 16), ('Eggplant pizza', 'A contemporary classic', '11.95', 16)]
Answer:All pizzas have a diameter of 16 inches for large, regardless of the type of pizza.
> Finished chain.


'All pizzas have a diameter of 16 inches for large, regardless of the type of pizza.'

In [35]:
# try again with sequential db chain, which checks what tables should first be examined
# not using query checker, but something is causing the sql to be much more accurate and
# the answers are correct

from langchain.chains import SQLDatabaseSequentialChain
seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

seq_chain.run("What is the price of a small cheese pizza topped with both sausage and mushroom?")



> Entering new  chain...
Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
What is the price of a small cheese pizza topped with both sausage and mushroom?
SQLQuery:SELECT p.Price_Small + t.Price + t2.Price AS Total_Price
FROM pizzas p, toppings t, toppings t2
WHERE p.Item = 'Cheese pizza' AND t.Item = 'Sausage' AND t2.Item = 'Mushrooms'
SQLResult: [(11.0,)]
Answer:The price of a small cheese pizza topped with both sausage and mushroom is $11.
> Finished chain.

> Finished chain.


'The price of a small cheese pizza topped with both sausage and mushroom is $11.'

In [36]:
seq_chain.run("How big in inches is each pizza size?")



> Entering new  chain...
Table names to use:
['pizzas']

> Entering new  chain...
How big in inches is each pizza size?
SQLQuery:SELECT "Item", "Price_Large" AS "Large (inches)", "Price_Medium" AS "Medium (inches)", "Price_Small" AS "Small (inches)" FROM pizzas
SQLResult: [('Pepperoni pizza', '12.95', '10', '7'), ('Cheese pizza', '10.95', '9.25', '6.50'), ('Eggplant pizza', '11.95', '9.75', '6.75')]
Answer:The large pizza is 12.95 inches, the medium pizza is 10 inches, and the small pizza is 7 inches. For Pepperoni pizza, the large pizza is 12.95 inches, the medium pizza is 10 inches, and the small pizza is 7 inches. For Cheese pizza, the large pizza is 10.95 inches, the medium pizza is 9.25 inches, and the small pizza is 6.50 inches. For Eggplant pizza, the large pizza is 11.95 inches, the medium pizza is 9.75 inches, and the small pizza is 6.75 inches.
> Finished chain.

> Finished chain.


'The large pizza is 12.95 inches, the medium pizza is 10 inches, and the small pizza is 7 inches. For Pepperoni pizza, the large pizza is 12.95 inches, the medium pizza is 10 inches, and the small pizza is 7 inches. For Cheese pizza, the large pizza is 10.95 inches, the medium pizza is 9.25 inches, and the small pizza is 6.50 inches. For Eggplant pizza, the large pizza is 11.95 inches, the medium pizza is 9.75 inches, and the small pizza is 6.75 inches.'

### Scratchwork - check LLM answers

In [24]:
cobj = dbio.connectors.SQLite('pizza_v2.db')

In [51]:
cobj.read("select * from drinks where Item like '%ater%'")

,Item,Price_Large,Price_Medium,Price_Small,Description
0,Bottled water,5.00,None,None,Single size = 20 ounces


In [49]:
# wrong answer for small cheese pizza + sausage + mushroom
# no columns returned

cobj.read('''
SELECT * -- "Price_Small" + SUM("Price") AS "Total Price"
FROM pizzas JOIN toppings ON pizzas.Item = toppings.Item
WHERE pizzas.Item = "Cheese pizza" AND toppings.Item IN ("Sausage", "Mushrooms")
''')

,Item,Price_Large,Price_Medium,Price_Small,Description,Item,Price,Description


In [48]:
# right answer for small cheese pizza + sausage + mushroom
# columns add to $11

cobj.read('''
SELECT p.Price_Small, t.Price, t2.Price  --p.Price_Small + t.Price + t2.Price AS Total_Price
FROM pizzas p, toppings t, toppings t2
WHERE p.Item = 'Cheese pizza' AND t.Item = 'Sausage' AND t2.Item = 'Mushrooms'
''')

,Price_Small,Price,Price
0,6.50,3.00,1.50


sqlDatabaseChain notes:
 - you can specify return_direct=True in the SQLDatabaseChain initialization to directly return the output of the SQL query without any additional formatting. This prevents the LLM from seeing any contents within the database. Note, however, the LLM still has access to the database scheme (i.e. dialect, table and key names) by default.
 - SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True) for query checking
 - Return intermediate steps instead of just the final answer: SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, use_query_checker=True, return_intermediate_steps=True)